In [9]:
import os
import pandas as pd
import re
import shutil
import uuid
import subprocess
import StringIO
import numpy as np
dssp_exec ='/opt/dssp/dssp'
pdb_path = '/mnt/Tsunami_HHD/Tang/onebyone_clsuter/muscle_alignment_pdb/3eqi.pdb'

a = subprocess.check_output([dssp_exec,pdb_path])
buf = StringIO.StringIO(a)
lines = buf.readlines()


name_idex  = lines.index('  #  RESIDUE AA STRUCTURE BP1 BP2  ACC     N-H-->O    O-->H-N    N-H-->O    O-->H-N    TCO  KAPPA ALPHA  PHI   PSI    X-CA   Y-CA   Z-CA \n')

lines = lines[name_idex+1:]

data = []
for line in lines:
    line_data = []
    line_data.append(line[6:10].strip())
    line_data.append(line[11:12].strip())
    line_data.append(line[13:14].strip())
    line_data.append(line[15:17].strip())
    line_data.append(line[18:25])
    line_data.append(line[26:29].strip())
    line_data.append(line[30:34].strip())
    line_data.append(line[35:38].strip())
    data.append(line_data)

Columns = ['resID',' chain','residue','type','structure','BP1','BP2','ACC']

DSSP_Data_Frame=pd.DataFrame(data,columns= Columns)
DSSP_Data_Frame.replace({'type':{'':'L'}},inplace = True)

secondary2score = {'H':0.5,'I':0.5,'G':0.5,'T':1,'S':1,'B':0,'E':0,'L':1}
assigneds = secondary2score.keys()
secondaryScore = DSSP_Data_Frame['type'].apply(lambda x: secondary2score[x] if x in assigneds else 1)
final_dssp_table = pd.DataFrame(zip(DSSP_Data_Frame['resID'],DSSP_Data_Frame['type'],secondaryScore),
                                columns=['resID','type','dssp_result'])

res_len = final_dssp_table.shape[0]

loop_num = sum (final_dssp_table['dssp_result'] == 1)

loop_percent = float (loop_num)/float (res_len) * float (100)
final_dssp_table['loop_percent'] = loop_percent
final_dssp_table['resID'] = final_dssp_table['resID'].replace('',np.nan, regex=True)

final_dssp_table = final_dssp_table[~final_dssp_table['resID'].isnull()]

In [10]:
final_dssp_table

,resID,type,dssp_result,loop_percent
0,1,L,1.0,39.490446
1,2,L,1.0,39.490446
2,3,L,1.0,39.490446
3,4,L,1.0,39.490446
4,5,L,1.0,39.490446
5,6,H,0.5,39.490446
6,7,H,0.5,39.490446
7,8,H,0.5,39.490446
8,9,H,0.5,39.490446
9,10,H,0.5,39.490446


In [5]:
DSSP_Data_Frame.replace({'type':{'':'L'}},inplace = True)

In [6]:
DSSP_Data_Frame

,resID,chain,residue,type,structure,BP1,BP2,ACC
0,1,A,E,L,,0,0,106
1,2,A,L,L,+,0,0,42
2,3,A,E,L,-,0,0,179
3,4,A,L,L,-,0,0,75
4,5,A,D,L,> -,0,0,105
5,6,A,E,H,> S+,0,0,177
6,7,A,Q,H,> S+,0,0,89
7,8,A,Q,H,> S+,0,0,34
8,9,A,R,H,X S+,0,0,129
9,10,A,K,H,X S+,0,0,119


In [ ]:
def getDSSP_final(pdbPATH):
    a = subprocess.check_output([dssp_exec,pdbPATH])
    buf = StringIO.StringIO(a)
    lines = buf.readlines()


    name_idex  = lines.index('  #  RESIDUE AA STRUCTURE BP1 BP2  ACC     N-H-->O    O-->H-N    N-H-->O    O-->H-N    TCO  KAPPA ALPHA  PHI   PSI    X-CA   Y-CA   Z-CA \n')

    lines = lines[name_idex+1:]

    data = []
    for line in lines:
        line_data = []
        line_data.append(line[6:10].strip())
        line_data.append(line[11:12].strip())
        line_data.append(line[13:14].strip())
        line_data.append(line[15:17].strip())
        line_data.append(line[18:25])
        line_data.append(line[26:29].strip())
        line_data.append(line[30:34].strip())
        line_data.append(line[35:38].strip())
        data.append(line_data)

    Columns = ['resID',' chain','residue','type','structure','BP1','BP2','ACC']

    DSSP_Data_Frame=pd.DataFrame(data,columns= Columns)


    secondary2score = {'H':0.5,'I':0.5,'G':0.5,'T':1,'S':1,'B':0,'E':0}
    assigneds = secondary2score.keys()
    secondaryScore = DSSP_Data_Frame['type'].apply(lambda x: secondary2score[x] if x in assigneds else 1)
    final_dssp_table = pd.DataFrame(zip(DSSP_Data_Frame['resID'],secondaryScore),columns=['resID','dssp_result'])
    res_len = final_dssp_table.shape[0]

    loop_num = sum (final_dssp_table['dssp_result'] == 1)

    loop_percent = float (loop_num)/float (res_len) * float (100)
    final_dssp_table['loop_percent'] = loop_percent
    final_dssp_table['resID'] = final_dssp_table['resID'].replace('',np.nan, regex=True)

    final_dssp_table = final_dssp_table[~final_dssp_table['resID'].isnull()]
    return final_dssp_table